# Jupyter Notebook to clean install all the contracts

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint

from repeater import retry

NONCE = 100
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

# Networks
Uncomment the line which you want to use.

In [ ]:
# Mainnet
# icon_service = IconService(HTTPProvider("https://ctz.solidwallet.io", 3))
# NID = 1

# Euljiro
# icon_service = IconService(HTTPProvider("https://test-ctz.solidwallet.io",3))
# NID = 2

# Yeouido
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
NID = 3

# Pagoda
# icon_service = IconService(HTTPProvider("https://zicon.net.solidwallet.io", 3))
# NID = 80

# Custom
# icon_service = IconService(HTTPProvider("http://127.0.0.1:9000/", 3))

# Load Wallet using Keystore file

In [ ]:
wallet = KeyWallet.load("./keystore/test1","passwordHere")
deployer_address = wallet.get_address()
deployer_address

# Load Wallet Using Private Key

In [ ]:
private="---PrivateKey-------"
prepwallet = KeyWallet.load(bytes.fromhex(private))
prep_address = wallet.get_address()
prep_address

# Check Wallet Balance

In [ ]:
icon_service.get_balance(deployer_address)

# Contracts

In [ ]:
#Load a pkl file if existed otherwise go to next step
contracts_file_custom = 'contracts_20201203030216.pkl'

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)
    
contracts

In [ ]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
# Run this before deploying scores 
contracts=AutoVivification()

# directory names
deploy = ['cps_score', 'CPSTreasury', 'CPFTreasury']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            contracts[file.name]['zip'] = archive_name 
contracts

# Deploying Contract

In [ ]:
#this cell needs to run only when you need to deploy new set of contracts
for score, loc in contracts.items():
    params = {}
    print(score,loc)

    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

# Updating a Contract

In [ ]:
# Contract Name
# 'cps_score', 'CPSTreasury', 'CPFTreasury'
contract = 'CPSTreasury'

# Generates an instance of transaction for deploying score
transaction = DeployTransactionBuilder() \
            .from_(wallet.get_address()) \
            .to(contracts[contract]["SCORE"]) \
            .nid(NID) \
            .nonce(NONCE) \
            .content_type("application/zip") \
            .content(gen_deploy_data_content(contracts[contract]["zip"])) \
            .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transaction, wallet,step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

# After Deploying Setups

## Add Admins Address (only Owner)

In [ ]:
# Add a wallet address 
params = {"_address":"hx2176d13280d465b68afbf7f9be758e966f654629"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("add_admin")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Remove Admin from Admin List

In [ ]:
#Remove the given wallet address from the admin list
params = {"_address":"hx2176d13280d465b68afbf7f9be758e966f654629"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("remove_admin")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

## Set CPS Treasury SCORE

In [ ]:
# Set CPS Treasury SCORE Address on CPS_SCORE and CPF Treasury

contracts_name = ['cps_score','CPFTreasury']

params = {"_score":contracts["CPSTreasury"]['SCORE']}

for contract in contracts_name:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[contract]['SCORE'])\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(NONCE)\
        .method("set_cps_treasury_score")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(tx_hash)

## Set CPF Treasury Score

In [ ]:
# Set CPF Treasury SCORE Address on CPS_SCORE and CPS Treasury

contracts_name = ['cps_score','CPSTreasury']

params = {"_score":contracts["CPFTreasury"]['SCORE']}

for contract in contracts_name:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[contract]['SCORE'])\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(NONCE)\
        .method("set_cpf_treasury_score")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(tx_hash)

## Set CPS Score

In [ ]:
# Set CPS_SCORE SCORE Address on CPF TREASURY and CPS Treasury

contracts_name = ['CPFTreasury','CPSTreasury']

params = {"_score":contracts["cps_score"]['SCORE']}

for contract in contracts_name:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[contract]['SCORE'])\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(NONCE)\
        .method("set_cps_score")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(tx_hash)

## Add Fund to CPF Treasury Score

In [ ]:
# Add some fund to the CPF Treasury Score (Maximum 1M ICX)

params = {}

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["CPFTreasury"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .value(200*10**18)\
    .method("add_fund")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

## Set Penalty Amounts

In [ ]:
# Penalty Amount : Amount for Re-registering from the denylist.
# set the penalty amount for the P-Reps who are on Denylist 
# 3 Tiers penalty system

# params = {"_penalty":['1000','1500','2500']}
params = {"_penalty":['10','20','30']}

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("set_prep_penalty_amount")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

## Set Initial BlockHeight

In [ ]:
# set a blockheight when the CPS to start from initial (only once)
params = {"_timestamp":"0x3"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("set_initialBlock")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

# 

## Main Methods

### Register P-Rep

In [ ]:
params = {}

transaction = CallTransactionBuilder()\
    .from_(prepwallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("register_prep")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, prepwallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Listing P-Reps Address

In [ ]:
## List the active registered prep with their details from System Score Address
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_PReps")\
                    .params({})\
                    .build()

result = icon_service.call(call)
result

### Submit a Proposal

In [ ]:
## Submit a proposal 
### params : _project_title(str) : Project title for the proposal
### total_budget(int) : Total Budget for the project (ICX amount)
### sponsor_address(Address) : Address of the Sponsor P-Rep (must be a registered P-Rep)
### ipfs_hash(str) : IPFS Hash key for the proposal submission
### ipfs_link(str) : IPFS Link for the submitted proposal (Can see the details filled up and submitted)
### project_duration(int) : Total project duration (months) (Maximum 6)

params ={"_proposals": {
			"project_title": "ICON Mobile Wallet",
			"total_budget": "100",
			"sponsor_address": "hxe59539154a3d77023943c3bbd5fc3a081d697e6a",
			"ipfs_hash": "bafybeid3ucflkuettyzvzbsnqhvw5mtqkgbtes4s5yykhr6vd4ma7f7aiy",
			"ipfs_link": "https://gateway.ipfs.io/ipfs/bafybeid3ucflkuettyzvzbsnqhvw5mtqkgbtes4s5yykhr6vd4ma7f7aiy",
			"project_duration": "4"
		}}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .value(50000000000000000000)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("submit_proposal")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Vote Proposal

In [ ]:
### Vote a Proposal (Voting Period)

params = {"_vote": "_approve/_reject/_abstain",
          "_vote_reason": "Vote Reason",
          "_ipfs_key": "Proposal IPFS Hash"}
transaction = CallTransactionBuilder()\
    .from_(sdpwallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("vote_proposal")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, sdpwallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Submit Progress Report

In [ ]:
## Submit a Progress Report 
### params : progress_report_title(str) : Progress Report title for the progress report
### ipfs_hash(str) : IPFS Hash key for the main proposal
### report_hash(str): IPFS Hash key for the progress report submission
### ipfs_link(str) : IPFS Link for the submitted progress report (Can see the details filled up and submitted)
### percentage_completed(int) : Approx. project completion percentage
### budget_adjustment(bool) : True if progress report has budget adjustment application to increase the budget for the project
### additional_budget(int) (ICX amount) : application for additional budget proposed
### additional_month(int) : Additional month required for the additional work proposed

params = {"_progress_report": {
                "report_hash": "Progress Report IPFS HASH Key",
                "ipfs_hash": "Proposal IPFS HASH key",
                "progress_report_title": "Progress Report Title",
                "budget_adjustment": "0/1",
                "additional_budget": "0",
                "ipfs_link": "IPFS Submission Link",
                "percentage_completed": "Current Project Completion Status(int)",
                "additional_month": "0"
            }
          }

transaction = CallTransactionBuilder()\
    .from_(prep_wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("submit_progress_report")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, prep_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Vote Progress Report

In [ ]:
### Vote a progress Report (Voting Period)
params = {"_vote": "_approve/_reject",
          "_vote_reason": "Reason For the Vote",
          "_ipfs_key": "Proposal IPFS HASH",
          "_report_key":"Progress Report IPFS Hash",
          "_budget_adjustment_vote":"_approve/_reject"}

transaction = CallTransactionBuilder()\
    .from_(sdpwallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("vote_progress_report")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, sdpwallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Update a period

In [ ]:
# To update a period from application period to voting period and vice-versa 
# after the block count of a period is completed
params = {}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("update_period")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

## Read Only Methods

In [ ]:
### check the wallet address status, also if they are on denylist the penalty amount to pay is also listed
params = {"_address":"hxb4e90a285a79687ec148c29faabe6f71afa8a066"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("login_prep")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

In [ ]:
## Get period status with currentBlock, NextBlock to change the period, remainingTime for the period, 
## Currently running period name and the total period span
params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_period_status")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

In [ ]:
### Get the list of progress report with their details of provided status maximum 50 entries at once, 20 by default

## "_status" :["_waiting","_approved","_rejected"]
params = {"_status":"_waiting"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_progress_reports")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

In [ ]:
## List the progress report for the provided Proposal Ipfs_key (project key)
params = {"_ipfs_key":"bafybeifrv2vxo7utazawsxw5aj2yw6w3gvp5cu26oa4tzy37l2az52v3pm"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_progress_reports_by_proposal")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

In [ ]:
## To check if the provided contributor has any progress report to submit in the current Application Period
params = {"_wallet_address":"hx2176d13280d465b68afbf7f9be758e966f654629"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_active_proposals")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

In [ ]:
## Get project amount and total amount pending for the given contributor address

params = {"_wallet_address":"hx2176d13280d465b68afbf7f9be758e966f654629"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["CPSTreasury"]['SCORE'])\
                    .method("get_contributor_projected_fund")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

In [ ]:
## Get project amount and total amount pending for the given sponsorAddress 

params = {"_wallet_address":"hxef35c447e3f657315a6990da0ddaf4c61fa146e4"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["CPSTreasury"]['SCORE'])\
                    .method("get_sponsor_projected_fund")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result